In [12]:
import pandas as pd
import numpy as np
import requests
import json
import pickle
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
import scipy
from flaskapp.models import Business
from flaskapp import db
from flaskapp.recommender import make_input, predict

In [13]:
restaurant_names = ['112 Eatery', '1st Wok', "50's Grill"]
user_ratings = [5.0, 5.0, 5.0]
rating_vector = make_input(restaurant_names, user_ratings)

In [15]:
prediction = predict(rating_vector)
sorted_indices = np.argsort(prediction)

In [17]:
sorted_indices[:20]

array([691, 214, 608, 670, 129, 375, 588, 461, 807, 244, 271, 605, 561,
       633, 750, 121,  51,  20, 465, 512])

In [5]:
names = pd.read_csv('data/names.csv')
for i in range(len(names)):
    name, category, coord = names.iloc[i, :]
    business = Business(name=name, coordinates=coord, categories=category)
    db.session.add(business)
db.session.commit()

In [6]:
df = pd.read_csv('data/business_data.csv')
df.head()

,Unnamed: 0,address,categories,coordinates,name,phone,price,query_offset,rating,restaurant_id,transactions,url,zip_code
0,50,NaN,"[{'alias': 'japanese', 'title': 'Japanese'}]","{'latitude': 44.921699931935, 'longitude': -93...",Okome House,NaN,NaN,1,5.0,6qn4Nt4frApZ2vn2rQerKg,[],https://www.yelp.com/biz/okome-house-minneapol...,55406.0
1,51,NaN,"[{'alias': 'korean', 'title': 'Korean'}]","{'latitude': 44.9819673599659, 'longitude': -9...",Kbop Korean Bistro,(612) 331-4993,$,1,4.5,vi58lz0GLWITkg_r0WieNg,"['pickup', 'delivery']",https://www.yelp.com/biz/kbop-korean-bistro-mi...,55414.0
2,52,NaN,"[{'alias': 'mediterranean', 'title': 'Mediterr...","{'latitude': 45.012975, 'longitude': -93.263755}",Olive & Lamb,NaN,$$,1,4.5,07VKEPjS7Juf2IB4jVa-wA,['delivery'],https://www.yelp.com/biz/olive-and-lamb-minnea...,55418.0
3,53,NaN,"[{'alias': 'newamerican', 'title': 'American (...","{'latitude': 44.9835380660176, 'longitude': -9...",Red Cow - Minneapolis,(612) 238-0050,$$,1,4.5,MDPv2B2jwA1qLvQCXlEVww,['delivery'],https://www.yelp.com/biz/red-cow-minneapolis-m...,55401.0
4,54,NaN,"[{'alias': 'korean', 'title': 'Korean'}]","{'latitude': 44.97916, 'longitude': -93.23471}",Korean Bowl,(612) 605-8960,NaN,1,5.0,VaEY4xoDh1Y5bvmvBb5Bug,"['pickup', 'delivery']",https://www.yelp.com/biz/korean-bowl-minneapol...,55414.0


In [42]:
df.drop_duplicates(subset='name', keep='first', inplace=True)

,categories,coordinates,name,phone,price,query_offset,rating,restaurant_id,transactions,url,zip_code
0,"[{'alias': 'japanese', 'title': 'Japanese'}]","{'latitude': 44.921699931935, 'longitude': -93...",Okome House,NaN,NaN,1,5.0,6qn4Nt4frApZ2vn2rQerKg,[],https://www.yelp.com/biz/okome-house-minneapol...,55406.0
1,"[{'alias': 'korean', 'title': 'Korean'}]","{'latitude': 44.9819673599659, 'longitude': -9...",Kbop Korean Bistro,(612) 331-4993,$,1,4.5,vi58lz0GLWITkg_r0WieNg,"['pickup', 'delivery']",https://www.yelp.com/biz/kbop-korean-bistro-mi...,55414.0
2,"[{'alias': 'mediterranean', 'title': 'Mediterr...","{'latitude': 45.012975, 'longitude': -93.263755}",Olive & Lamb,NaN,$$,1,4.5,07VKEPjS7Juf2IB4jVa-wA,['delivery'],https://www.yelp.com/biz/olive-and-lamb-minnea...,55418.0
3,"[{'alias': 'newamerican', 'title': 'American (...","{'latitude': 44.9835380660176, 'longitude': -9...",Red Cow - Minneapolis,(612) 238-0050,$$,1,4.5,MDPv2B2jwA1qLvQCXlEVww,['delivery'],https://www.yelp.com/biz/red-cow-minneapolis-m...,55401.0
4,"[{'alias': 'korean', 'title': 'Korean'}]","{'latitude': 44.97916, 'longitude': -93.23471}",Korean Bowl,(612) 605-8960,NaN,1,5.0,VaEY4xoDh1Y5bvmvBb5Bug,"['pickup', 'delivery']",https://www.yelp.com/biz/korean-bowl-minneapol...,55414.0


In [46]:
names = df.loc[:, ['name', 'categories', 'coordinates']]
names.to_csv('data/names.csv', index=False)


,name,categories,coordinates
0,Okome House,"[{'alias': 'japanese', 'title': 'Japanese'}]","{'latitude': 44.921699931935, 'longitude': -93..."
1,Kbop Korean Bistro,"[{'alias': 'korean', 'title': 'Korean'}]","{'latitude': 44.9819673599659, 'longitude': -9..."
2,Olive & Lamb,"[{'alias': 'mediterranean', 'title': 'Mediterr...","{'latitude': 45.012975, 'longitude': -93.263755}"
3,Red Cow - Minneapolis,"[{'alias': 'newamerican', 'title': 'American (...","{'latitude': 44.9835380660176, 'longitude': -9..."
4,Korean Bowl,"[{'alias': 'korean', 'title': 'Korean'}]","{'latitude': 44.97916, 'longitude': -93.23471}"


In [15]:
df['coordinates'].apply(lambda x: (json.loads(x)['latitude'], json.loads(x)['longitude']))

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [22]:
names = df.loc[:, ['names', 'coordinates', 'categories']]
names.to_csv('data/names.csv')

In [21]:
test

"{'latitude': 44.921699931935, 'longitude': -93.2123801484704}"

In [13]:
model = LightFM(loss='warp', no_components=50)
model.fit(train, epochs=100)
train_precision = precision_at_k(model, train, k=10).mean()
print(train_precision)

NameError: name 'train' is not defined

In [12]:
model.predict(rating_vector, np.arange(names.shape[0]))

array([-8.56973417e-03, -1.30643237e+00,  1.57228932e-02, -9.37740624e-01,
       -5.25542259e-01,  1.36807486e-01, -3.30875784e-01, -8.70892346e-01,
        6.99218288e-02, -2.00041324e-01, -1.09470963e+00, -9.62633193e-01,
       -3.02075297e-01, -7.55709112e-01, -4.38898087e-01,  1.71446875e-01,
        5.89263588e-02, -3.35353017e-01, -1.22458446e+00, -5.53000510e-01,
       -1.45270228e+00, -8.71211469e-01, -9.16200757e-01, -9.60602582e-01,
       -1.06396258e+00, -2.31011912e-01, -4.02503908e-01, -5.50684333e-01,
       -1.26531637e+00, -5.87008953e-01, -4.42767024e-01,  1.20969124e-01,
       -6.01991832e-01, -1.30081952e+00, -1.03799200e+00, -5.53155184e-01,
       -7.74758816e-01, -3.94998699e-01, -5.37457287e-01, -2.70655781e-01,
       -3.05632651e-01, -6.38454556e-01, -1.35597038e+00, -7.05816865e-01,
       -1.14701581e+00, -3.08248490e-01, -4.56547767e-01, -3.20211977e-01,
        2.08617508e-01, -1.44268274e-01, -5.27988732e-01, -1.45498407e+00,
       -8.58255148e-01, -

In [7]:
rating_vector = np.zeros(names.shape[0])
sorted_index = np.searchsorted(names, restaurant_names)
for i, res_name in enumerate(restaurant_names):
    if names[sorted_index[i]].lower() == res_name.lower():
        rating_vector[sorted_index[i]] = user_ratings[i]
        print(f'set {res_name} to {user_ratings[i]}')


set 112 Eatery to 5.0
set 1st Wok to 5.0
set 50's Grill to 5.0


In [15]:
filename = 'data/recommender'
pickle.dump(model, open(filename, 'wb'))
  
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))


In [17]:
train_precision = precision_at_k(loaded_model, train, k=10).mean()
print(train_precision)

0.14249086


In [4]:
review_df = pd.read_csv('data/full_review_data.csv')
review_df.drop_duplicates(subset = ['user_id', 'restaurant_id'], keep='first', inplace=True)
review_df = review_df[~review_df['restaurant_name'].isnull()]
bus_df = pd.read_csv('data/business_data.csv')
bus_df = bus_df[bus_df['name'].isin(review_df['restaurant_name'])]
bus_df.drop_duplicates(subset='name', keep='first', inplace=True)
bus_df.sort_values(by='name', inplace=True)
review_df = review_df[['user_id', 'restaurant_name', 'rating']]
pivoted = pd.pivot(review_df, index='user_id', columns='restaurant_name', values='rating')
pivoted.fillna(value=0, inplace=True)
item_df = process_item_features(bus_df)

In [9]:
bool_dict = {False: 0, True: 1}
item_df['has_pickup'] = item_df['has_pickup'].map(lambda x: bool_dict[x])
item_df['has_delivery'] = item_df['has_delivery'].map(lambda x: bool_dict[x])

In [11]:
item_sparse = scipy.sparse.coo_matrix(item_df)

In [12]:
item_sparse

<819x4 sparse matrix of type '<class 'numpy.float64'>'
	with 2424 stored elements in COOrdinate format>

In [13]:
pivoted.head()

restaurant_name,$5 Pizza,112 Eatery,1st Wok,50's Grill,5th Street Station,6300 Steakhouse,801 Chophouse,8th Street Grill & Taphouse,A Slice of New York,A&J Fish & Chicken,...,Yum! Kitchen and Bakery,Zait & Za'atar,Zakia Deli,Zelino Deli,Zelo,Zettas,Zoe's Cafe,Zumbro Cafe,iPierogi,ie Italian Eatery
user_id,,,,,,,,,,,,,,,,,,,,,
-2LdRY6wGWVdBa83R9-M-A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-3GuMSvftRhN2yn1TFIWEw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-3aLm3nt7dX2KnEyHsONig,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-68rYG6a5YtxGI0pCevZmQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-6aUbeI5oyxAfhnqWq26zA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0


In [39]:
# Define the recommender system
# To-do: train the recommender system, save the model


In [33]:
pivoted.head()

restaurant_name,$5 Pizza,112 Eatery,1st Wok,50's Grill,5th Street Station,6300 Steakhouse,801 Chophouse,8th Street Grill & Taphouse,A Slice of New York,A&J Fish & Chicken,...,Yum! Kitchen and Bakery,Zait & Za'atar,Zakia Deli,Zelino Deli,Zelo,Zettas,Zoe's Cafe,Zumbro Cafe,iPierogi,ie Italian Eatery
user_id,,,,,,,,,,,,,,,,,,,,,
-2LdRY6wGWVdBa83R9-M-A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-3GuMSvftRhN2yn1TFIWEw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-3aLm3nt7dX2KnEyHsONig,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-68rYG6a5YtxGI0pCevZmQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-6aUbeI5oyxAfhnqWq26zA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0


In [34]:
# Convert review_df to training matrix
review_df = pd.read_csv('data/full_review_data.csv')
review_df.drop_duplicates(subset = ['user_id', 'restaurant_id'], keep='first', inplace=True)
df = review_df[['user_id', 'restaurant_name', 'rating']]
pivoted = pd.pivot(df, index='user_id', columns='restaurant_name', values='rating')
pivoted.head()
pivoted.fillna(value=0, inplace=True)
sparse = scipy.sparse.csr_matrix(pivoted.values)
np.savez_compressed('data/train.npz',  train=sparse)

In [41]:
bus_df = pd.read_csv('data/business_data.csv')
bus_df.head()

,address,categories,coordinates,name,phone,price,query_offset,rating,restaurant_id,transactions,url,zip_code
0,NaN,"[{'alias': 'japanese', 'title': 'Japanese'}]","{'latitude': 44.921699931935, 'longitude': -93...",NaN,NaN,NaN,1,5.0,6qn4Nt4frApZ2vn2rQerKg,[],https://www.yelp.com/biz/okome-house-minneapol...,55406.0
1,NaN,"[{'alias': 'korean', 'title': 'Korean'}]","{'latitude': 44.9819673599659, 'longitude': -9...",NaN,(612) 331-4993,$,1,4.5,vi58lz0GLWITkg_r0WieNg,"['delivery', 'pickup']",https://www.yelp.com/biz/kbop-korean-bistro-mi...,55414.0
2,NaN,"[{'alias': 'mediterranean', 'title': 'Mediterr...","{'latitude': 45.012975, 'longitude': -93.263755}",NaN,NaN,$$,1,4.5,07VKEPjS7Juf2IB4jVa-wA,['delivery'],https://www.yelp.com/biz/olive-and-lamb-minnea...,55418.0
3,NaN,"[{'alias': 'newamerican', 'title': 'American (...","{'latitude': 44.9835380660176, 'longitude': -9...",NaN,(612) 238-0050,$$,1,4.5,MDPv2B2jwA1qLvQCXlEVww,['delivery'],https://www.yelp.com/biz/red-cow-minneapolis-m...,55401.0
4,NaN,"[{'alias': 'korean', 'title': 'Korean'}]","{'latitude': 44.97916, 'longitude': -93.23471}",NaN,(612) 605-8960,NaN,1,5.0,VaEY4xoDh1Y5bvmvBb5Bug,"['delivery', 'pickup']",https://www.yelp.com/biz/korean-bowl-minneapol...,55414.0


In [81]:
# Preprocessing the restaurant-level features
bus_df = pd.read_csv('data/business_data.csv')
item_features = bus_df.loc[:, ['price', 'rating', 'transactions']]
item_features.index = bus_df['name']
item_features = item_features[~item_features.index.isnull()]
item_features['has_pickup'] = item_features['transactions'].apply(lambda li: 'pickup' in li)
item_features['has_delivery'] = item_features['transactions'].apply(lambda li: 'delivery' in li)
item_features.drop('transactions', axis=1, inplace=True)
def price_mapping(s):
    d = {np.nan: np.nan, '$':1, '$$': 2, '$$$': 3, '$$$$': 4}
    return d[s]
item_features['price'] = item_features['price'].map(price_mapping)
item_features.head()

,price,rating,has_pickup,has_delivery
name,,,,
Okome House,NaN,5.0,False,False
Kbop Korean Bistro,1.0,4.5,True,True
Olive & Lamb,2.0,4.5,False,True
Red Cow - Minneapolis,2.0,4.5,False,True
Korean Bowl,NaN,5.0,True,True


In [79]:
item_features.head()

,55104.0,55105.0,55108.0,55111.0,55112.0,55113.0,55114.0,55116.0,55126.0,55150.0,...,55439.0,55441.0,55454.0,55455.0,55456.0,55467.0,has_delivery,has_pickup,price,rating
name,,,,,,,,,,,,,,,,,,,,,
Okome House,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,NaN,5.0
Kbop Korean Bistro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,True,True,1.0,4.5
Olive & Lamb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,True,False,2.0,4.5
Red Cow - Minneapolis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,True,False,2.0,4.5
Korean Bowl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,True,True,NaN,5.0


In [67]:
#item_features = pd.concat([item_features, pd.get_dummies(item_features['zip_code'])])
item_features.columns

Index([       55104.0,        55105.0,        55108.0,        55111.0,
              55112.0,        55113.0,        55114.0,        55116.0,
              55126.0,        55150.0,        55305.0,        55343.0,
              55344.0,        55401.0,        55402.0,        55403.0,
              55404.0,        55405.0,        55406.0,        55407.0,
              55408.0,        55409.0,        55410.0,        55411.0,
              55412.0,        55413.0,        55414.0,        55415.0,
              55416.0,        55417.0,        55418.0,        55419.0,
              55421.0,        55422.0,        55423.0,        55424.0,
              55426.0,        55427.0,        55428.0,        55429.0,
              55430.0,        55435.0,        55439.0,        55441.0,
              55454.0,        55455.0,        55456.0,        55467.0,
         'categories', 'has_delivery',   'has_pickup',        'price',
             'rating',     'zip_code'],
      dtype='object')

In [9]:
df = pd.read_csv('data/business_data.csv')
df = df[~df.name.isnull()]
df.head()

,address,categories,coordinates,name,phone,price,query_offset,rating,restaurant_id,transactions,url,zip_code
50,NaN,"[{'alias': 'japanese', 'title': 'Japanese'}]","{'latitude': 44.921699931935, 'longitude': -93...",Okome House,NaN,NaN,1,5.0,6qn4Nt4frApZ2vn2rQerKg,[],https://www.yelp.com/biz/okome-house-minneapol...,55406.0
51,NaN,"[{'alias': 'korean', 'title': 'Korean'}]","{'latitude': 44.9819673599659, 'longitude': -9...",Kbop Korean Bistro,(612) 331-4993,$,1,4.5,vi58lz0GLWITkg_r0WieNg,"['pickup', 'delivery']",https://www.yelp.com/biz/kbop-korean-bistro-mi...,55414.0
52,NaN,"[{'alias': 'mediterranean', 'title': 'Mediterr...","{'latitude': 45.012975, 'longitude': -93.263755}",Olive & Lamb,NaN,$$,1,4.5,07VKEPjS7Juf2IB4jVa-wA,['delivery'],https://www.yelp.com/biz/olive-and-lamb-minnea...,55418.0
53,NaN,"[{'alias': 'newamerican', 'title': 'American (...","{'latitude': 44.9835380660176, 'longitude': -9...",Red Cow - Minneapolis,(612) 238-0050,$$,1,4.5,MDPv2B2jwA1qLvQCXlEVww,['delivery'],https://www.yelp.com/biz/red-cow-minneapolis-m...,55401.0
54,NaN,"[{'alias': 'korean', 'title': 'Korean'}]","{'latitude': 44.97916, 'longitude': -93.23471}",Korean Bowl,(612) 605-8960,NaN,1,5.0,VaEY4xoDh1Y5bvmvBb5Bug,"['pickup', 'delivery']",https://www.yelp.com/biz/korean-bowl-minneapol...,55414.0


In [13]:
df.drop_duplicates(subset='name', keep='first', inplace=True)
df.shape

(822, 12)

In [14]:
name = df['name'].values[0]
id_key = df['restaurant_id'].values[0]
print(f'name: {name}, id_key: {id_key}')
#review_response = query_review_api(id_key, api_key)
review_df = make_review_df(review_response['reviews'], id_key, name)
review_df.head()
#all_reviews = pd.concat([all_reviews, review_df])

name: Okome House, id_key: 6qn4Nt4frApZ2vn2rQerKg


,rating,review,timestamp,review_id,user_name,user_id,restaurant_id,restaurant_name
0,5,What a darling neighborhood spot! Lots of take...,2020-09-03 19:42:33,lIfGtbTIs0iFY-FO2fTeXg,Teri M.,qsvF7iQ8kHKFqDo9HccYLA,6qn4Nt4frApZ2vn2rQerKg,Okome House
1,5,Love the veggie curry and chicken karaage! Tas...,2020-09-29 08:08:39,glv_p4Ja9yFaIMF1rP4utg,Tiger H.,NJVkWqKTtaR3w9zOMl-zAw,6qn4Nt4frApZ2vn2rQerKg,Okome House
2,5,We have been waiting for this restaurant to op...,2020-08-29 17:32:02,6pwyJnB024AP2z9K4O1YsA,Maggie B.,W08sB7PohxU0z3csKM8ihg,6qn4Nt4frApZ2vn2rQerKg,Okome House


In [38]:
review_df.to_csv('data/full_review_data.csv', index=False)

In [36]:
for i in range(389,len(df)):
    name, id_key = df['name'].values[i], df['restaurant_id'].values[i]
    review_response = query_review_api(id_key, api_key)
    try:
        new_reviews = make_review_df(review_response['reviews'], id_key, name)
    except:
        print(f'stopped loop at {name}, i = {i}')
        time.sleep(5)
        continue
    review_df = pd.concat([review_df, new_reviews])
    time.sleep(1)

stopped loop at Caffrey's Deli & Subs, i = 492


In [40]:
review_df.head(50)

,rating,review,timestamp,review_id,user_name,user_id,restaurant_id,restaurant_name
0,5,What a darling neighborhood spot! Lots of take...,2020-09-03 19:42:33,lIfGtbTIs0iFY-FO2fTeXg,Teri M.,qsvF7iQ8kHKFqDo9HccYLA,6qn4Nt4frApZ2vn2rQerKg,Okome House
1,5,Love the veggie curry and chicken karaage! Tas...,2020-09-29 08:08:39,glv_p4Ja9yFaIMF1rP4utg,Tiger H.,NJVkWqKTtaR3w9zOMl-zAw,6qn4Nt4frApZ2vn2rQerKg,Okome House
2,5,We have been waiting for this restaurant to op...,2020-08-29 17:32:02,6pwyJnB024AP2z9K4O1YsA,Maggie B.,W08sB7PohxU0z3csKM8ihg,6qn4Nt4frApZ2vn2rQerKg,Okome House
0,5,This is the BEST Korean food place in MN! Even...,2020-08-15 17:13:26,MJETH4iL5OFnuhZ3AdW_5A,Abbey C.,gleOsBEFdHGQUJu9p0lijA,vi58lz0GLWITkg_r0WieNg,Kbop Korean Bistro
1,5,Love this place so much! I work close by so it...,2020-07-12 12:35:56,LtBzaUimLaJeOEzoNkinAw,Anila Y.,Cz_I-lkHk4wBQx7i3UN04A,vi58lz0GLWITkg_r0WieNg,Kbop Korean Bistro
2,5,This place takes social distancing very seriou...,2020-07-20 16:59:24,jCLad87USD9j7JI0bqnHqA,Lidiya L.,k2lHZN8uJ84SrD2z85rcWA,vi58lz0GLWITkg_r0WieNg,Kbop Korean Bistro
0,5,This is the best Mediterranean food I have had...,2020-10-11 12:38:13,qOMOAExF9XOTFGbNSMM9xg,Sarah K.,TjQxvguf_kR3fQ8W5_OHzg,07VKEPjS7Juf2IB4jVa-wA,Olive & Lamb
1,5,Very cute little place with amazing food! We h...,2020-08-24 14:48:54,KV8NZrIQi26lnJrsX8y4Og,Taya B.,zZUEQ6H7_cr4exhnhro2ZQ,07VKEPjS7Juf2IB4jVa-wA,Olive & Lamb
2,5,One of my favorite Mediterranean restaurants! ...,2020-09-26 15:42:20,nVd-JCOX2GTw5IR9g5dpZQ,Sarah S.,5UgTAODbiJ1P9Qo5V_8X4A,07VKEPjS7Juf2IB4jVa-wA,Olive & Lamb
0,5,I am so glad this restaurant is operating duri...,2020-07-09 14:52:31,d9FB8U7ciSeP6LPQGK-PBw,Linda S.,1_i42G-exZoyBdCirzYWBQ,MDPv2B2jwA1qLvQCXlEVww,Red Cow - Minneapolis


In [48]:
review_df = pd.read_csv('data/review_data.csv')
len(review_df)



149

In [5]:
review_list = review_response['reviews']
make_review_df(review_list)

,rating,review,timestamp,restaurant_id,user_name,user_id
0,5,What a darling neighborhood spot! Lots of take...,2020-09-03 19:42:33,lIfGtbTIs0iFY-FO2fTeXg,Teri M.,qsvF7iQ8kHKFqDo9HccYLA
1,5,Love the veggie curry and chicken karaage! Tas...,2020-09-29 08:08:39,glv_p4Ja9yFaIMF1rP4utg,Tiger H.,NJVkWqKTtaR3w9zOMl-zAw
2,5,We have been waiting for this restaurant to op...,2020-08-29 17:32:02,6pwyJnB024AP2z9K4O1YsA,Maggie B.,W08sB7PohxU0z3csKM8ihg


In [4]:
data = [(elt['name'], elt['rating'], elt['location']['address1'], elt['location']['zip_code'], elt['display_phone'], elt['url'], elt['id'], elt['price'], elt['transactions'], elt['categories'], elt['coordinates']) for elt in business_data['businesses']]


KeyError: 'price'

In [6]:
business_data['businesses'][-2]

{'id': 'WvgAuJo6rUM1_I2sVDEzyg',
 'alias': 'snack-bar-minneapolis',
 'name': 'Snack Bar',
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/0ti5h9qhtyM-5fPphGuiBQ/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/snack-bar-minneapolis?adjust_creative=5q2gXrFINpdsi56KQ3uWdQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=5q2gXrFINpdsi56KQ3uWdQ',
 'review_count': 39,
 'categories': [{'alias': 'pizza', 'title': 'Pizza'},
  {'alias': 'tapasmallplates', 'title': 'Tapas/Small Plates'}],
 'rating': 4.0,
 'coordinates': {'latitude': 44.9896, 'longitude': -93.2786408},
 'transactions': [],
 'location': {'address1': '800 N Washington Ave',
  'address2': '',
  'address3': None,
  'city': 'Minneapolis',
  'zip_code': '55401',
  'country': 'US',
  'state': 'MN',
  'display_address': ['800 N Washington Ave', 'Minneapolis, MN 55401']},
 'phone': '+16123832848',
 'display_phone': '(612) 383-2848',
 'distance': 2826.192739302379}

In [5]:
reader = Reader(
    line_format='user item rating', sep=',', rating_scale = (0,5)
    )
train_df = pd.read_csv('train.csv')
data = Dataset.load_from_df(train_df, reader=reader)
trainset, testset = train_test_split(data, test_size=0.2)
#trainsetfull = data.build_full_trainset()

In [76]:
my_k = 15
my_min_k = 5
my_sim_option = {
    'name':'pearson', 'user_based':False}
algo = KNNWithMeans(k = my_k, min_k = my_min_k, sim_option = my_sim_option)
algo.fit(trainset)

predictions = algo.test(testset)
accuracy.rmse(predictions)

60

In [ ]:
results = cross_validate(
    algo = algo, data = data, measures=['RMSE'], 
    cv=5, return_train_measures=True
    )
    results['test_rmse'].mean()

In [77]:
businesses = pd.read_csv('business_data.csv')

In [79]:
businesses.drop_duplicates(keep='first', inplace=True)
len(businesses)

20

In [84]:
from flaskapp.download_reviews import query_business_api




    

ModuleNotFoundError: No module named 'flaskapp'

In [80]:
retrieved_data = query_business_api(offset=1).json()

NameError: name 'query_business_api' is not defined

In [68]:

reviews = pd.read_csv('review_data.csv', index_col=0)
reviews.head(20)

,rating,review,timestamp,restaurant_id,user_name,user_id
0,5,"Fuck, you guys. Everything here will melt and ...",2020-10-07 19:09:09,D799KG1dx-Vf3zpZ5JreSQ,Annalee T.,iqS1mA7zdFEUIjz8KW5oVw
1,5,I miss this restaurant more than most during C...,2020-09-19 09:02:22,Sx78U_0np04cZwBYUIDZ4g,Ursula B.,hHeyv-uAGMn3pyaswmkzrg
2,4,This was date night for me and the girlfriend ...,2020-06-12 23:44:35,QswObuJ7dLFadPl_iPJcgA,Jack G.,ftaFFF1-9md-nhm7QqhfrQ
0,5,Always a delicious meal at 112 Eatery. All th...,2020-10-16 06:59:55,KG7lVQvWQbBN4p7B7XMlhg,Joyce I.,ttCKlOKicxWLchdl1qsMgQ
1,4,The food: always amazing; as it has been for 1...,2020-06-30 07:15:23,MYCNWMGp0e9iVN6I6SUIRg,Carter A.,IyDE1kaZu4Ho5skVxe4Jmg
2,3,I really loved it. Always a delicious meal at ...,2020-10-17 10:58:36,vcyNp7DivsNth-8NikBh9g,Great Dane P.,WP7BAr6ssVaCvvIt_BKyYQ
0,5,Saw George and the Dragon featured on DDD and ...,2020-08-14 19:00:29,hia3_i2OcZj_18v9LZ-wfg,Brandon J.,Sg8jj6BO8AoGlKz54lHh8Q
1,5,"Still amazing, pandemic or no. Really everythi...",2020-08-21 11:18:38,0vUzpY5hZ7LB7N09yICwdg,S And B D.,2LgAn3XIW_wYubsTs3r-kQ
2,5,What a treat. Flew into town and wanted to fi...,2020-08-18 16:57:54,pMQOGhmtJsOXlH5z5kUZyg,Ben L.,w_Ru_isrLBtbdtxro5TYEg
0,5,We went during COVID so I'm sure we didn't get...,2020-07-14 06:06:14,p0Q7Capy-TtY8pfvCCG0nQ,Dale G.,Jljum8MVQ2l067RzRqWJqQ


In [43]:
review_df.to_csv('review_data.csv')

In [2]:
df = pd.read_csv("Food_Inspections.csv")
df['BusinessName'] = df['BusinessName'].str.upper()
df['BusinessName'] = df['BusinessName'].str.strip()
categories = ['RESTAURANT', 'FOOD TRUCK', 'FOOD CART']
df = df[df['FacilityCategory'] .isin(categories)]
df.drop_duplicates(subset = ['BusinessName'], keep='last', inplace=True)
keep_cols = ['BusinessName', 'X', 'Y', 'FullAddress', 'City', 'ZipCode', 'Neighborhood', 'Latitude', 'Longitude']
df = df.loc[:, keep_cols]

In [27]:
df.to_csv('restaurants.csv')

In [24]:
df[df['FacilityCategory'] == 'FOOD CART']


,X,Y,OBJECTID,HealthFacilityIDNumber,FacilityCategory,BusinessName,RiskLevel,FullAddress,AddrNum,AddrNumSuffix,...,ViolationPoints,InspectionScore,ViolationIDNumber,APN,Ward,Neighborhood,Latitude,Longitude,X_WebMercator,Y_WebMercator
27178,-93.275787,44.977720,27179,LIC38173,FOOD CART,TOP DOGS,2,722 HENNEPIN AVE,722.0,NaN,...,4,94,677467,2202924430133,7.0,Downtown West,44.97772,-93.27579,-1.038341e+07,5.618015e+06
21450,0.000000,0.000000,21451,LIC365072,FOOD CART,PIONEER COLLEGE CATERERS,2,1401 ELLIOT AVE,1401.0,NaN,...,0,100,-1,NaN,NaN,NaN,0.00000,0.00000,0.000000e+00,0.000000e+00
32510,-93.234367,45.004833,32511,LIC42319,FOOD CART,UNCLE FRANKY'S TOOL BOX,2,1520 NEW BRIGHTON BLVD,1520.0,NaN,...,0,100,-1,1302924120016,1.0,Northeast Park,45.00483,-93.23437,-1.037880e+07,5.622282e+06
10985,-93.262172,44.974589,10986,LIC32948,FOOD CART,HENN CTY ADULT SERV-VOC SERV PROGR,2,525 PORTLAND AVE #300,525.0,NaN,...,0,100,-1,2602924210053,7.0,Elliot Park,44.97459,-93.26217,-1.038190e+07,5.617522e+06
27079,-93.307592,44.991771,27080,LIC381380,FOOD CART,MR. FAULKNER'S OLD FASHIONED HOT DOGS,1,2100 PLYMOUTH AVE N,2100.0,NaN,...,0,100,-1,1602924339000,5.0,Willard - Hay,44.99177,-93.30759,-1.038695e+07,5.620226e+06
27168,-93.273850,44.975640,27169,LIC38170,FOOD CART,TOP DOG,1,800 NICOLLET MALL,800.0,NaN,...,0,100,-1,2702924120139,7.0,Downtown West,44.97564,-93.27385,-1.038320e+07,5.617687e+06
56965,-93.273850,44.975640,56966,LIC45072,FOOD CART,NIBBLES,2,800 NICOLLET MALL,800.0,NaN,...,2,92,770547,2702924120139,7.0,Downtown West,44.97564,-93.27385,-1.038320e+07,5.617687e+06


In [36]:
def binary_search(A, start, end, target):
    mid = start + (end-start)//2
    if A[mid] >= target:
        if end <= start + 1:
            return mid
        return binary_search(A, start, mid, target)
    else:
        if end <= start + 1:
            return end
        return binary_search(A, mid+1, end, target)

In [38]:
A = [1,2,3,4,5,6,7,8,9]
print(binary_search(A, 0, len(A)-1, 8))

7
